In [465]:
# Import libraries
import pandas as pd
import folium
import ast
from branca.utilities import split_six
from branca.colormap import linear
from folium import plugins
from folium.plugins import FloatImage

In [466]:
df = pd.read_csv("./results/merged_11_20.csv")

# We are interested only in the maus
df = df[df["Frequency"] == "Monthly"]  # Daily, Monthly
df = df[df["LocationHierarchy"] == "Country"]
# Filter specific locations to plot
df = df[df["Location"].apply(lambda x: x in ["Brazil", "Colombia", "Chile", "Ecuador", "Argentina", "Uruguay", "Peru", "Panama"])]


In [473]:

newcols = {"general": {"validation": "valid", "incol": "venezuelans", "outcol":"venPopRatio", "numerator": "allpopulation", "cmtitle": "% of Venezuelans"},
           "13-18": {"validation": "validAdolecent", "incol": "audience_adolecent", "outcol":"venAdolecentRatio", "numerator": "venezuelans", "cmtitle": "% of Venezuelans (<18yo)"},
            "19-25": {"validation": "validYAdult", "incol": "audience_young_adult", "outcol":"venYAdultRatio", "numerator": "venezuelans", "cmtitle": "% of Venezuelans (<19-25yo)"},
            "26-40": {"validation": "validAdult", "incol": "audience_adult", "outcol":"venAdultRatio", "numerator": "venezuelans", "cmtitle": "% of Venezuelans (26-40yo)"},
            "40-65": {"validation": "validMiddleAge", "incol": "audience_middle_age", "outcol":"venMiddleAgeRatio", "numerator": "venezuelans", "cmtitle": "% of Venezuelans (40-65yo)"},
            ">65": {"validation": "validElder", "incol": "audience_elder", "outcol":"venElderRatio", "numerator": "venezuelans", "cmtitle": "% of Venezuelans (>65yo)"},
    
            "android": {"validation": "validAndroid", "incol": "audience_Android", "outcol":"androidRatio", "numerator": "venezuelans", "cmtitle": "% of Venezuelans (Android)"},
            "iOS": {"validation": "validiOS", "incol": "audience_iOS", "outcol":"iOSRatio", "numerator": "venezuelans", "cmtitle": "% of Venezuelans (iOS)"},
           
            "men": {"validation": "validGender", "incol": "audience_man", "outcol":"menRatio", "numerator": "venezuelans", "cmtitle": "% of Men from Venezuelans"},
          } 

for k, g in newcols.items():
    df[g["validation"]] = False
    df.loc[df[g["incol"]] > 1000, g["validation"]] = True

df["validGender"] = False
df.loc[(df['venezuelans'] > 1000) & (df['audience_man'] > 1000) & (df['audience_woman'] > 1000), "validGender"] = True

def rationize(df, colname, validcol, col_den, col_num, sumover=False):
    df[colname] = np.nan
    if sumover:
        df.loc[df[validcol], colname] = 1. * \
            df[col_den] / (df[df[validcol]][col_num].sum())
    else:
        df.loc[df[validcol], colname] = 1. * df[col_den] / df[col_num]
    df[colname] = df[colname].fillna(-1)

rationize(df, "venPopDist", "valid", "venezuelans", "venezuelans", sumover=True)
rationize(df, "venMigrantsRatio", "valid", "venezuelans", "allmigrants")

for k, g in newcols.items():
    rationize(df, g["outcol"], g["validation"], g["incol"], g["numerator"])


In [474]:
import branca.colormap as cm

step = cm.StepColormap(
    ['green', 'yellow', 'orange', 'red'],
    vmin=0, vmax=1500000,
    index=[0, 10000, 50000, 100000, 1000000],
    caption='Number of Venezuelans'
)

linear_0_10 = cm.LinearColormap(['green', 'yellow', 'red'],vmin=0., vmax=0.1)
linear_0_20 = cm.LinearColormap(['green', 'yellow', 'red'],vmin=0., vmax=0.2)
linear_0_30 = cm.LinearColormap(['green', 'yellow', 'red'],vmin=0., vmax=0.3)
linear_40_60 = cm.LinearColormap(['green', 'yellow', 'red'],vmin=0.4, vmax=0.6)
linear_0_40 = cm.LinearColormap(['green', 'yellow', 'red'],vmin=0.0, vmax=0.4)

# Change this colormap as you like
colormap = cm.linear.GnBu_05

df["color_venezuelans"] = df["venezuelans"].apply(lambda x : step(x))
df["color_venPopRatio"] = df["venPopRatio"].apply(lambda x : colormap(x))
df["color_venMigrantsRatio"] = df["venMigrantsRatio"].apply(lambda x : colormap(x))
df["color_androidRatio"] = df["androidRatio"].apply(lambda x : linear_0_20(x))
df["color_iOSRatio"] = df["iOSRatio"].apply(lambda x : linear_0_20(x))
df["color_menRatio"] = df["menRatio"].apply(lambda x : linear_40_60(x))
df["color_venPopDist"] = df["venPopDist"].apply(lambda x : colormap(x))
#df["color_venAdolecent"] = df["venAdolecent"].apply(lambda x : linear_0_10(x))
#df["color_venAdult"] = df["venAdultRatio"].apply(lambda x : linear_40_60(x))
#df["color_venYA"] = df["venYARatio"].apply(lambda x : linear_0_40(x))
#df["color_venMA"] = df["venMARatio"].apply(lambda x : linear_0_20(x))
#df["color_venElderRatio"] = df["venElderRatio"].apply(lambda x : linear_0_20(x))


#Set1_03, GnBu_05, Paired_10, YlOrRd_04, Paired_06, StepColormap(['green', 'yellow', 'orange', 'red'])
for k, g in newcols.items():
    colorset = cm.LinearColormap(['green', 'yellow', 'red'])
    newcols[k]["colormap"] = colorset.scale(np.max([0.0, df[g["outcol"]].replace(-1, np.nan).min().round(2)]), df[g["outcol"]].max().round(2)) # .to_step(5)
    newcols[k]["colormap"].caption = g["cmtitle"]
    df["color_" + g["outcol"]] = df[g["outcol"]].apply(lambda x : g["colormap"](x))
    



In [478]:
# Json with shapefiles of brazil
state_geo = "countries.json"
 
# Initialize the map:
#m = folium.Map(location=[-25, -50], zoom_start=4, tiles='Mapbox Bright') # tiles='openstreetmap'
m = folium.Map(location=[-25, -97], zoom_start=4, tiles='Mapbox Bright') # tiles='openstreetmap'

g = newcols["men"]

fg = folium.FeatureGroup("Venezuelan Population", overlay=True)
folium.GeoJson(data=state_geo,
               name='# Venezuelans',smooth_factor=1,
               style_function=lambda feature: {
                        'fillColor': 'black' if not df[df["Location"] == feature['id']][g["validation"]].values[0] else df[df["Location"] == feature['id']]["color_" + g["outcol"]].values[0],
                        'color': 'black',
                        'weight': 3,
                        'dashArray': '5, 5',
                        'fillOpacity': 0.6,
                },
               highlight_function=lambda x: {'weight':3, 'fillColor':'grey'}
              ).add_to(fg)
m.add_child(fg)
g["colormap"].add_to(m)

#folium.LayerControl().add_to(m)

url = ("https://www.qcri.org/app/media/3480")
FloatImage(url, bottom=8, left=81).add_to(m)

# Save to html
#m.save('br.html')

m

In [228]:
# Json with shapefiles of brazil
state_geo = "states_Brazil.json"
 
# Load the unemployment value of each state
# Find the original file here: https://github.com/python-visualization/folium/tree/master/examples/data

#col = "100ven/pop"
#col = "venezuelans"
#threshold_scale = split_six(df[col])

# Initialize the map:
m = folium.Map(location=[-15, -55], zoom_start=4.5, tiles='Mapbox Bright')

# Add the color for the chloropleth:
m.choropleth(
 geo_data=state_geo,
 name='Venezuelans',
 data=df,
 columns=['Location', "venezuelans"],
 key_on='feature.id',
 fill_color='YlGn',
 #fill_color='BuPu',
 fill_opacity=0.7,
 line_opacity=0.2,
 legend_name='# Venezuelans',
)

def getValue(df, location, col="venezuelans"):
    return df[df["Location"] == location][col].values[0]

def getHTML(df, location):
    man = getValue(df, location, "audience_man")
    woman = getValue(df, location, "audience_woman")

    html = """
    <h3> %s </h3><br>
    <p>
        Men: %d / Women: %d
    </p>
    """ % (location, man, woman)
    return html


folium.Marker(
    location=[-4.941829, -64.750795],
    popup=getHTML(df, "Amazonas, Brazil")
).add_to(m)

folium.Marker(
    location=[1.845650,-61.856050],
    popup=getHTML(df, "Roraima, Brazil")
).add_to(m)

folium.Marker(
    location=[-18.462676,-44.547864],
    popup=getHTML(df, "Minas Gerais, Brazil")
).add_to(m)

folium.Marker(
    location=[-23.550520,-46.633308],
    popup=getHTML(df, "São Paulo, Brazil")
).add_to(m)

folium.Marker(
    location=[-22.906847,-43.1728978],
    popup=getHTML(df, "Rio de Janeiro, Brazil")
).add_to(m)

folium.Marker(
    location=[-29.167391,-51.180061],
    popup=getHTML(df, "Rio Grande do Sul, Brazil")
).add_to(m)

folium.Marker(
    location=[-6.643780,-36.868300],
    popup=getHTML(df, "Pernambuco, Brazil")
).add_to(m)



folium.LayerControl().add_to(m)

from folium.plugins import FloatImage
url = ("https://www.qcri.org/app/media/3480")
FloatImage(url, bottom=5, left=75).add_to(m)
 
# Save to html
m.save('br.html')
m


IndexError: index 0 is out of bounds for axis 0 with size 0

In [87]:
geo = pd.read_json(state_geo)
geo["name"] = geo["features"].apply(lambda x: x["id"])
#geo["features"].apply(lambda x: x["geometry"]["coordinates"])


In [102]:
import pandas as pd

n_periods, n_sample = 30, 27

datetime_index = pd.date_range('2016-1-1', periods=n_periods, freq='M')
dt_index_epochs = datetime_index.astype(int) // 10**9
dt_index = dt_index_epochs.astype('U10')

dt_index

Index(['1454198400', '1456704000', '1459382400', '1461974400', '1464652800',
       '1467244800', '1469923200', '1472601600', '1475193600', '1477872000',
       '1480464000', '1483142400', '1485820800', '1488240000', '1490918400',
       '1493510400', '1496188800', '1498780800', '1501459200', '1504137600',
       '1506729600', '1509408000', '1512000000', '1514678400', '1517356800',
       '1519776000', '1522454400', '1525046400', '1527724800', '1530316800'],
      dtype='object')

In [112]:
styledata = {}

for location in geo["name"]:
    df = pd.DataFrame(
        {'color': np.random.normal(size=n_periods),
         'opacity': np.random.normal(size=n_periods)},
        index=dt_index
    )
    df = df.cumsum()
    df.sample(n_sample, replace=True).sort_index()
    styledata[location] = df

max_color, min_color, max_opacity, min_opacity = 0, 0, 0, 0

for country, data in styledata.items():
    max_color = max(max_color, data['color'].max())
    min_color = min(max_color, data['color'].min())
    max_opacity = max(max_color, data['opacity'].max())
    max_opacity = min(max_color, data['opacity'].max())
    
from branca.colormap import linear


cmap = linear.PuRd_09.scale(min_color, max_color)

def norm(x):
    return (x - x.min()) / (x.max() - x.min())

for country, data in styledata.items():
    data['color'] = data['color'].apply(cmap)
    data['opacity'] = norm(data['opacity'])
    
styledict = {
    str(country): data.to_dict(orient='index') for
    country, data in styledata.items()
}


In [117]:
# https://nbviewer.jupyter.org/github/python-visualization/folium/blob/master/examples/TimeSliderChoropleth.ipynb
from folium.plugins import TimeSliderChoropleth

# Initialize the map:
m = folium.Map(location=[-15, -55], zoom_start=5, tiles='Mapbox Bright')

g = TimeSliderChoropleth(
    state_geo,
    styledict=styledict,
).add_to(m)

m

In [30]:
m = folium.Map(location=[-15, -55], zoom_start=4.5, tiles='Mapbox Bright')

fg = folium.FeatureGroup("Whatever")
m.add_child(fg)

g1 = plugins.FeatureGroupSubGroup(fg, 'Men')
m.add_child(g1)

g2 = plugins.FeatureGroupSubGroup(fg, 'Women')
m.add_child(g2)

folium.GeoJson(data=state_geo,
               name='Men',smooth_factor=2,
               style_function=lambda feature: {
                        'fillColor': colormaps["man"](man_dict[feature['id']]),
                        'color': 'black',
                        'weight': 2,
                        'dashArray': '5, 5'
                },
               #style_function=lambda x: {'color':'black','fillColor':'transparent','weight':2},
               tooltip=folium.GeoJsonTooltip(fields=['name'], labels=False, sticky=False),
               highlight_function=lambda x: {'weight':3,'fillColor':'grey'}
              ).add_to(g1)


folium.Marker([-1,-1]).add_to(g1)
folium.Marker([1,1]).add_to(g1)

folium.Marker([-1,1]).add_to(g2)
folium.Marker([1,-1]).add_to(g2)



l = folium.LayerControl().add_to(m)

m

In [130]:

m = folium.Map(location=[2.88, -60.77], zoom_start=16, tiles='openstreetmap')



folium.CircleMarker(
    location=[2.8889, -60.7762],
    radius=2,
    fill=True,
    popup=folium.Popup('inline explicit Popup')
).add_to(m)

m